In [1]:
!python --version

Python 3.9.6


# Genetic algorithm implementation

In [2]:
from typing import List, Optional, Tuple
import random
from random import uniform, randrange
import math

In [19]:
# genetic algorithm parameters
POPULATION_SIZE: int = 100
GENOME_LENGHT: int = 10
VALUES_RANGE: Tuple[float, float] = (-5.0, 5.0)
CROSSOVER_PERCENTAGE: float = 0.3
SURVIVORS_PERCENTAGE: float = 0.1
MUTATION_PROBABILITY: float = 0.1

```
class Gene {
  public Float value;
  public Gene(Float value) { this.value = value; }
  public static Gene get_random(Pair<Float, Float> values_range) {
    random_value = RandomFloat(values_range.getValue0(), values_range.getValue1();
    return new Gene(random_value);}}
```

In [4]:
class Gene:  # == 'trait'/'feature'
    def __init__(self, value: float):
        self.value: float = value
    @classmethod
    def get_random(cls):
        random_value = uniform(VALUES_RANGE[0], VALUES_RANGE[1])
        return cls(random_value)

In [5]:
# class Genome {
#   public ArrayList<Gene> genes;
#   public Genome(ArrayList<Gene> genes) { this.genes = genes; }}

class Genome:  # == 'set of traits/features'
    def __init__(self, genes: List[Gene]):
        self.genes: List[Gene] = genes
    @classmethod
    def get_random(cls):
        genes: List[Gene] = []
        for gene_index in range(GENOME_LENGHT):
            genes.append(Gene.get_random())
        return cls(genes)
    def crossover(self, other_genome):
        result_genes: List[Gene] = []
        for gene_index in range(len(self.genes)):
            if uniform(0.0, 1.0) < 0.5:
                result_genes.append(Gene(self.genes[gene_index].value))
            else:
                result_genes.append(Gene(other_genome.genes[gene_index].value))
        return Genome(result_genes)
    def mutate(self):  # here could be multiple types of mutations e.g. shift by value from normal distribution
        if uniform(0.0, 1.0) < MUTATION_PROBABILITY:
            mutating_gene_index = randrange(len(self.genes))
            self.genes[mutating_gene_index].value = uniform(VALUES_RANGE[0], VALUES_RANGE[1])
    def __str__(self):
        genes_values = [gene.value for gene in self.genes]
        return str(genes_values)

In [6]:
# class Individual {
#   public Genome genome;
#   public Float score;
#   public Individual(Genome genome) { this.genome = genome }}

class Individual:  # 'genome representative'
    def __init__(self, genome: Genome):
        self.genome: Genome = genome
        self.score: Optional[float] = None
    @classmethod
    def get_random(cls):
        genome = Genome.get_random()
        return cls(genome)

In [7]:
# class Population {
#   public ArrayList<Individual> individuals;
#   public Population(ArrayList<Individual> individuals) { this.individuals = individuals }}

class Population:  # 'set of Individuals'
    def __init__(self, individuals: List[Individual]):
        self.individuals: List[Individual] = individuals
    def get_the_best(self) -> Individual:
        self.individuals.sort(key=lambda individual:individual.score, reverse=True)
        return self.individuals[0]
    def get_top_scored(self, percentage: float) -> List[Individual]:
        n_best: int = math.ceil(len(self.individuals) * percentage)
        return self.individuals[:n_best]

In [8]:
def generate_init_population():
    generated_individuals: List[Individual] = []
    for individual_index in range(POPULATION_SIZE):
        generated_individuals.append(Individual.get_random())
    return Population(generated_individuals)

In [9]:
def generate_children(individuals: List[Individual], children_number: int) -> List[Individual]:
    children: List[Individual] = []
    for child_index in range(children_number):
        parent1: Individual = random.choice(individuals)
        individuals_minus_parent1 = individuals.copy()
        individuals_minus_parent1.remove(parent1)
        parent2: Individual = random.choice(individuals_minus_parent1)
        new_genome = parent1.genome.crossover(parent2.genome)
        new_genome.mutate()
        children.append(Individual(new_genome))
    return children

In [16]:
class ProblemAbstractClass:
    def __init__(self, target_score: float, iterations_limit: int):
        self.target_score: float = target_score
        self.iterations_limit: int = iterations_limit
        # generate initial population
        self.current_population: Population = generate_init_population()
            
    # THIS IS THE GENETIC ALGORITHM
    def solve(self) -> Individual:
        # rate individuals
        self.rate_individuals()
        iteration: int = 0
        # if stop condition is met - return the best individual from current_population
        current_the_best = self.current_population.get_the_best()
        while current_the_best.score < self.target_score and iteration < self.iterations_limit:
            # choose individuals for crossover
            crossover_individuals: List[Individual] = self.current_population.get_top_scored(CROSSOVER_PERCENTAGE)
            # get the best individuals from current_population
            survivors: List[Individual] = self.current_population.get_top_scored(SURVIVORS_PERCENTAGE)
            # generate children by crossover and mutation
            children: List[Individual] = generate_children(crossover_individuals, POPULATION_SIZE - len(survivors))
            # create new population as children plus the best individuals from current_population
            self.current_population = Population(children + survivors)
            # rate individuals
            self.rate_individuals()
            
            current_the_best = self.current_population.get_the_best()
            iteration += 1
            print("Iteration: " + str(iteration))
            print("Best genome: " + str(current_the_best.genome))
            print("Best score: " + str(current_the_best.score))
        return self.current_population.get_the_best()
    
    def rate_individuals(self):
        for individual in self.current_population.individuals:
            if individual.score is None:  # if it's not a survivor (avoid rating multiple times)
                individual.score = self.rate_one_individual(individual)
                
    def rate_one_individual(self, individual: Individual) -> float:
        # override this method:
        # 1. Interpret individual's genome as potential solution
        # 2. Confront it with the problem
        # 3. Return score for this solution/individual/genome
        pass

# Solving problem

In [17]:
class MaximizeSum(ProblemAbstractClass):
    def rate_one_individual(self, individual: Individual) -> float:
        score = 0
        for gene in individual.genome.genes:
            score += gene.value
        return score

In [23]:
problem_instance = MaximizeSum(49.8, 500)
problem_instance.solve()

Iteration: 1
Best genome: [3.3812891368736473, 4.10434545498876, 3.823197769566386, -3.38295062347147, 3.404833308071524, 4.708699123224395, 4.726522653974506, 4.4735777910848675, 1.7083636352423444, 0.3162460652843846]
Best score: 27.264124314839346
Iteration: 2
Best genome: [3.5648975922355888, 4.300604401654585, 2.2205286923669876, 4.671075307882264, 0.807288528355957, 4.2901580729645055, 4.13900379413044, 2.140644495624091, 2.445244742088458, 0.3162460652843846]
Best score: 28.89569169258726
Iteration: 3
Best genome: [3.3812891368736473, 4.300604401654585, 2.2205286923669876, 4.175633611374961, 3.404833308071524, 1.712971837404548, 4.726522653974506, 4.4735777910848675, 3.7724420620424155, 4.668153789321547]
Best score: 36.83655728416959
Iteration: 4
Best genome: [3.3812891368736473, 4.300604401654585, 2.2205286923669876, 4.671075307882264, 3.404833308071524, 1.712971837404548, 4.726522653974506, 4.4735777910848675, 3.7724420620424155, 4.668153789321547]
Best score: 37.331998980676

Best score: 49.28783575300625
Iteration: 156
Best genome: [4.975500733994746, 4.978199941705547, 4.9286451034015855, 4.972818899562535, 4.995872066263109, 4.93966243772544, 4.726522653974506, 4.989311911700513, 4.8326164650517995, 4.948685539626467]
Best score: 49.28783575300625
Iteration: 157
Best genome: [4.975500733994746, 4.978199941705547, 4.9286451034015855, 4.972818899562535, 4.995872066263109, 4.93966243772544, 4.726522653974506, 4.989311911700513, 4.8326164650517995, 4.948685539626467]
Best score: 49.28783575300625
Iteration: 158
Best genome: [4.975500733994746, 4.978199941705547, 4.9286451034015855, 4.972818899562535, 4.995872066263109, 4.93966243772544, 4.726522653974506, 4.989311911700513, 4.8326164650517995, 4.948685539626467]
Best score: 49.28783575300625
Iteration: 159
Best genome: [4.975500733994746, 4.978199941705547, 4.9286451034015855, 4.972818899562535, 4.995872066263109, 4.93966243772544, 4.726522653974506, 4.989311911700513, 4.8326164650517995, 4.948685539626467]


Iteration: 301
Best genome: [4.998601969841189, 4.978199941705547, 4.999283383190065, 4.990381559961371, 4.996252101911436, 4.993548392874686, 4.97254516978292, 4.989311911700513, 4.904768043809272, 4.948685539626467]
Best score: 49.77157801440346
Iteration: 302
Best genome: [4.998601969841189, 4.978199941705547, 4.999283383190065, 4.990381559961371, 4.996252101911436, 4.993548392874686, 4.97254516978292, 4.989311911700513, 4.904768043809272, 4.948685539626467]
Best score: 49.77157801440346
Iteration: 303
Best genome: [4.998601969841189, 4.978199941705547, 4.999283383190065, 4.990381559961371, 4.996252101911436, 4.993548392874686, 4.97254516978292, 4.989311911700513, 4.904768043809272, 4.948685539626467]
Best score: 49.77157801440346
Iteration: 304
Best genome: [4.998601969841189, 4.978199941705547, 4.999283383190065, 4.990381559961371, 4.996252101911436, 4.993548392874686, 4.97254516978292, 4.989311911700513, 4.904768043809272, 4.948685539626467]
Best score: 49.77157801440346
Iteratio